In [ ]:
from jira import JIRA
import re

# Jira credentials
username = ""
password = ""
url = ""

# Create a Jira connection
jira = JIRA(server=url, basic_auth=(username, password))

In [ ]:
def parse_scds_ticket(issue):
    try:
        summary = issue.raw["fields"]["summary"]
        status = issue.raw["fields"]["status"]["name"]

        if issue.raw["fields"].get("customfield_10140"):
            analyst = issue.raw["fields"]["customfield_10140"][0]["displayName"]
        else:
            analyst = "Unassigned"

        parent_key = issue.raw["fields"]["parent"]["key"]
        # get an issue by name
        parent_issue = jira.issue(parent_key)
        epic_batch_name = parent_issue.raw["fields"]["parent"]["fields"]["summary"]

        return {
            "summary": summary,
            "status": status,
            "analyst": analyst,
            "epic_batch_name": epic_batch_name,
        }
    except Exception as ex:
        return


def parse_batch(s):
    pattern = re.compile(r"batch(\d+)")
    match = pattern.search(s)
    batch_number = match.group(1)
    return batch_number


def parse_shortname(s):
    pattern = re.compile(r".*deliverables/(.*)-README.txt")
    match = pattern.search(s)
    batch_number = match.group(1)
    return batch_number


def parse_dataset(s):
    pattern = re.compile(r"/data/batch\d/(.*)/deliverables.*")
    match = pattern.search(s)
    batch_number = match.group(1)
    return batch_number


def compose_name(shortname, dataset, batch_no):
    return f"{shortname} ({dataset}) batch{batch_no} var table patch"


def create_issue_in_epic(
    title, description=None, parent_key="CURATIONBD-1071", project_key="CURATIONBD"
):
    # Issue details
    issue_data = {
        "project": {"key": project_key},
        "summary": title,
        "description": description,
        "issuetype": {"name": "Task"},
    }

    # Create the issue
    new_issue = jira.create_issue(fields=issue_data)

    # Link to the epic
    new_issue.update(fields={"parent": {"key": parent_key}})

    print(f"Issue created successfully. Key: {new_issue.key}")

In [ ]:
def parse_epic_tickets(batch_name = 'Batch 8', epic_name):
    issues = jira.search_issues(
        "project = CURATIONBD"
        " AND issuetype = Subtask"
        f' AND "Batch[Dropdown]" = "{batch_name}"'
        ' AND "Workflow[Dropdown]" = Bioinformatics'
        " AND status not in (Blocked)",
        maxResults=None,
    )
    # parse issues
    parsed_issues = [parse_scds_ticket(issue) for issue in issues]
    # remove missing issues
    parsed_issues = [issue for issue in parsed_issues if issue]
    # filter to just one epic batch
    parsed_issues = [
        issue for issue in parsed_issues if issue["epic_batch_name"] == epic_name
    ]
    return parsed_issues

def status_report(parsed_issues):
    # list all statuses
    status = {
        "To Do": 0,
        "4. Bio Download": 0,
        "5. Bio Processing": 0,
        "6. Bio formatting": 0,
        "6.1 Bio QC": 0,
        "7. Delivery QC": 0,
        "Done": 0,
    }

    report = {}

    for issue in parsed_issues:
        this_issue = issue
        this_analyst = this_issue["analyst"]

        if this_analyst not in report:
            report[this_analyst] = status.copy()
        report[this_analyst][this_issue["status"]] += 1

    return report


In [ ]:
# parse_epic_tickets("Year 2: Batch 8b Datasets")